In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import AdaBoostRegressor

# Open the [Ames Housing Dataset ](https://www.kaggle.com/datasets/thedevastator/predicting-credit-card-customer-attrition-with-m?select=BankChurners.csv)

In [2]:
#Open the housing data and split into training/test data
houses = pd.read_csv('AmesHousing.csv')
prices = houses['SalePrice']

#Remove the PID and order columns
houses.drop(['PID'], axis=1, inplace=True)
houses.drop(['Order'], axis=1, inplace=True)

# Exploring the data

In [3]:
houses.columns

Index(['MS SubClass', 'MS Zoning', 'Lot Frontage', 'Lot Area', 'Street',
       'Alley', 'Lot Shape', 'Land Contour', 'Utilities', 'Lot Config',
       'Land Slope', 'Neighborhood', 'Condition 1', 'Condition 2', 'Bldg Type',
       'House Style', 'Overall Qual', 'Overall Cond', 'Year Built',
       'Year Remod/Add', 'Roof Style', 'Roof Matl', 'Exterior 1st',
       'Exterior 2nd', 'Mas Vnr Type', 'Mas Vnr Area', 'Exter Qual',
       'Exter Cond', 'Foundation', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure',
       'BsmtFin Type 1', 'BsmtFin SF 1', 'BsmtFin Type 2', 'BsmtFin SF 2',
       'Bsmt Unf SF', 'Total Bsmt SF', 'Heating', 'Heating QC', 'Central Air',
       'Electrical', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF',
       'Gr Liv Area', 'Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath',
       'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr', 'Kitchen Qual',
       'TotRms AbvGrd', 'Functional', 'Fireplaces', 'Fireplace Qu',
       'Garage Type', 'Garage Yr Blt', 'Garage Finish', 'Gara

In [4]:
print('There are '+ str(houses.shape[1]-1)+' total features, excluding sale price.')

There are 79 total features, excluding sale price.


In [5]:
min_year_sold = houses['Yr Sold'].min()
max_year_sold = houses['Yr Sold'].max()

print('The min year for which a house was sold is '+str(min_year_sold)+' and the last year for which a house was sold in this dataset was '+
     str(max_year_sold))

The min year for which a house was sold is 2006 and the last year for which a house was sold in this dataset was 2010


# There are missing or NA entries for several of the columns

For the Alley column, the possible entries are
 'Grvl' Gravel
 'Pave' Paved
 'NA' No alley access
  Most of the entries for alley are 'NA'. We drop this column.

In [6]:
print(houses['Alley'].isna().sum())

#Drop alley
houses.drop(['Alley'], axis=1, inplace=True)

2732


In [7]:
missing = []
for col in houses.columns:
    c=houses[col].isna().sum()
    if c>0 and col!='Alley':
        print(col)
        missing.append(col)

Lot Frontage
Mas Vnr Type
Mas Vnr Area
Bsmt Qual
Bsmt Cond
Bsmt Exposure
BsmtFin Type 1
BsmtFin SF 1
BsmtFin Type 2
BsmtFin SF 2
Bsmt Unf SF
Total Bsmt SF
Electrical
Bsmt Full Bath
Bsmt Half Bath
Fireplace Qu
Garage Type
Garage Yr Blt
Garage Finish
Garage Cars
Garage Area
Garage Qual
Garage Cond
Pool QC
Fence
Misc Feature


# Data Cleaning. For categorical variables, we convert 'NA' entries to the string 'NA'. For numerical features, we convert NaN entries to the mean of that column.

In [8]:
def cleanString(data,column):
    data[column].fillna('NA',inplace=True)
    return 
  
def cleanNumerical(data,column):
    mean = data[column].mean()
    
    data[column].fillna(mean,inplace=True)

def clean(data,column):
    if data[column].dtype=='O':
        cleanString(data,column)
    else:
        cleanNumerical(data,column)


In [9]:
for col in missing:
    clean(houses, col)

# Next, we use one hot encoding to convert categorical variables into numerical variables.
We include Mo Sold and MS SubClass, which are numerical variables. MS SubClass identifies the type of dwelling.
MS Subclass has discrete values ranging from 20 (1-story 1946 & newer all styles) to 190 (2 family conversion- all styles and ages)

In [10]:
one_hot_variables=['Mo Sold', 'MS SubClass']
for col in houses.columns:
    if houses[col].dtype=='O':
        one_hot_variables.append(col)

houses = pd.get_dummies(houses, columns = one_hot_variables)
        

In [11]:
#Display the 20 features that are most correlated with sale price
houses.corr()['SalePrice'].sort_values()[-21:houses.shape[1]-1]

Exter Qual_Gd         0.446489
Heating QC_Ex         0.454553
BsmtFin Type 1_GLQ    0.455326
Fireplaces            0.474558
Exter Qual_Ex         0.480477
TotRms AbvGrd         0.495474
Mas Vnr Area          0.505784
Garage Yr Blt         0.510684
Foundation_PConc      0.520966
Year Remod/Add        0.532974
Kitchen Qual_Ex       0.537561
Full Bath             0.545604
Year Built            0.558426
Bsmt Qual_Ex          0.593567
1st Flr SF            0.621676
Total Bsmt SF         0.632105
Garage Area           0.640385
Garage Cars           0.647861
Gr Liv Area           0.706780
Overall Qual          0.799262
Name: SalePrice, dtype: float64

In [12]:
#Drop sale price column
houses.drop(['SalePrice'], axis=1, inplace=True)

#80/20 Split into training and test data

X_train, X_test, y_train, y_test = train_test_split(houses, prices, test_size=.2)

# Use PCA transformation for dimensionality reduction

In [13]:
#Display the correlation matrix
houses.corr()

,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,...,Sale Type_New,Sale Type_Oth,Sale Type_VWD,Sale Type_WD,Sale Condition_Abnorml,Sale Condition_AdjLand,Sale Condition_Alloca,Sale Condition_Family,Sale Condition_Normal,Sale Condition_Partial
Lot Frontage,1.000000,0.365503,0.198900,-0.067383,0.115688,0.085744,0.203383,0.199946,0.040161,0.107733,...,0.115477,-0.003721,0.013673,-0.085191,0.003067,-0.046550,0.007688,0.015972,-0.083478,0.113294
Lot Area,0.365503,1.000000,0.097188,-0.034759,0.023258,0.021682,0.126586,0.191546,0.083145,0.023657,...,0.032328,-0.003937,0.008495,-0.015027,-0.014079,-0.020244,0.024450,0.018232,-0.024052,0.034169
Overall Qual,0.198900,0.097188,1.000000,-0.094812,0.597027,0.569609,0.427470,0.284011,-0.041272,0.269956,...,0.336153,-0.052845,-0.014340,-0.191898,-0.127738,-0.049768,-0.041004,-0.024062,-0.131968,0.331010
Overall Cond,-0.067383,-0.034759,-0.094812,1.000000,-0.368773,0.047680,-0.135088,-0.050920,0.041122,-0.136780,...,-0.147645,-0.018507,0.007263,0.161921,-0.059856,-0.051729,-0.025606,-0.031880,0.167683,-0.143081
Year Built,0.115688,0.023258,0.597027,-0.368773,1.000000,0.612095,0.311828,0.279836,-0.027412,0.128983,...,0.343270,-0.013985,-0.008161,-0.203243,-0.120844,-0.077805,-0.025238,-0.036261,-0.137162,0.338843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Sale Condition_AdjLand,-0.046550,-0.020244,-0.049768,-0.051729,-0.077805,-0.078199,-0.018507,-0.028191,-0.018855,-0.006412,...,-0.019111,-0.003138,-0.001185,0.025277,-0.016887,1.000000,-0.005828,-0.008099,-0.138542,-0.019371
Sale Condition_Alloca,0.007688,0.024450,-0.041004,-0.025606,-0.025238,-0.034022,-0.017823,0.025815,-0.026720,-0.061049,...,-0.027083,-0.004447,-0.001679,0.035820,-0.023931,-0.005828,1.000000,-0.011477,-0.196332,-0.027452
Sale Condition_Family,0.015972,0.018232,-0.024062,-0.031880,-0.036261,-0.049793,-0.017750,-0.026518,-0.002098,0.024427,...,-0.037638,0.050057,-0.002334,-0.014601,-0.033257,-0.008099,-0.011477,1.000000,-0.272844,-0.038150
Sale Condition_Normal,-0.083478,-0.024052,-0.131968,0.167683,-0.137162,-0.102775,-0.070109,-0.016431,0.069015,-0.151087,...,-0.643836,-0.087383,0.008553,0.620601,-0.568899,-0.138542,-0.196332,-0.272844,1.000000,-0.652596


In [14]:
#Scale training features to have mean 0 and variance 1

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [15]:
#Use PCA transform on X_train, and store transformation

PCA = PCA(n_components=200)
PCA.fit(X_train)

X_train_pca = PCA.transform(X_train)
X_test_pca = PCA.transform(X_test)

In [16]:
print('Using the PCA transform, we go from '+str(X_train.shape[1])+' features to '+str(X_train_pca.shape[1])+' features')

Using the PCA transform, we go from 343 features to 200 features


# Model 0: Baseline classification using mean of prices

In [17]:
#Compute mean of prices and create constant array of length prices having entries equal to mean
mean = prices.mean()
const_mean_array = pd.Series(data=[mean]*len(prices))

print(mean)

180796.0600682594


The mean of the prices is equal to 180796.06. For model 0, we just classifying every house as having price equal to this mean.

In [18]:
rmse = mean_squared_error(const_mean_array, prices, squared=False)

print('Model 0 has a RMSE of '+str(rmse)+' on the housing data')

Model 0 has a RMSE of 79873.05865192253 on the housing data


# Model 1: Linear regression

In [19]:
#Fit X_train to y_train

model1 = LinearRegression().fit(X_train_pca, y_train )

In [20]:
train_rmse = mean_squared_error(model1.predict(X_train_pca), y_train, squared=False)
test_rmse = mean_squared_error(model1.predict(X_test_pca), y_test, squared=False)
print('Model 1 has a RMSE of '+str(train_rmse)+' on the training data and '+str(test_rmse)+' on the test data.')

Model 1 has a RMSE of 23318.07000890546 on the training data and 33022.460069188586 on the test data.


# Model 2: Regression tree with AdaBoost


In [21]:
model2 = AdaBoostRegressor(n_estimators=11).fit(X_train_pca, y_train) 

In [22]:
train_rmse = mean_squared_error(model2.predict(X_train_pca), y_train, squared=False)
test_rmse = mean_squared_error(model2.predict(X_test_pca), y_test, squared=False)
print('Model 2 has a RMSE of '+str(train_rmse)+' on the training data and '+str(test_rmse)+' on the test data.')

Model 2 has a RMSE of 34629.22023584677 on the training data and 38116.94619720778 on the test data.
